In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import json

 # Yelp Training Data

In [3]:
res_mer = pd.read_csv('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/yelp.train.rating', sep ='\t')

In [4]:
res_mer.head()

,user_id,business_id,rating,date
0,0,0,3,2015-03-19
1,0,1,3,2015-03-19
2,0,2,4,2015-03-19
3,0,3,4,2015-03-19
4,0,4,3,2015-03-19


# Utility Matrix Creation

In [5]:
df = res_mer.pivot(index = 'user_id', columns ='business_id', values='rating')

In [6]:
rating_mat = df.as_matrix().astype(np.float)

# Centralizing each row vector to mean zero

In [7]:
arrMean = np.nanmean(rating_mat, axis = 1)

In [8]:
arrMean = arrMean[:, np.newaxis]

In [9]:
mat = rating_mat - arrMean

In [10]:
mat[np.isnan(mat)] = 0

In [11]:
print("Number of users:", len(mat))
print("Number of items:", len(mat[0]))

Number of users: 19771
Number of items: 48114


# Similarity Matrix

In [12]:
def sim_matrix(u,eps=1.0e-9):
    step1 = np.dot(u, u.T) + eps
    step2 = np.array([np.sqrt(np.diagonal(step1))])
    return (step1 / step2 / step2.T)

 We added small value so that it should not give divide by zero error if zero would be there.

# Top K Similar Items

In [13]:
def top_k(m_utility, item_idx, arr, self_idx, k):
    sortedArr=arr.argsort()[::-1][:len(arr)]
    sortedArr=sortedArr[1:k+1]
    tk={}
    for i in sortedArr: 
        if i != self_idx:
            tk[i]=arr[i]
    return tk

# Recommendation via Similar Users

In [14]:
def rec_via_users(m_utility, m_sim_users, user_idx, item_idx, k):
    dict_s = top_k(m_utility, item_idx, m_sim_users[user_idx], user_idx, k)
    s = 0.00
    c = 0.00
    for k,v in dict_s.items():
        if item_idx > m_utility.shape[1]: # if it is new item, we will give it neutral rating
            return 0
        if m_utility[k][item_idx] == 0: #continuing if user has not rated that item
            continue
        s += (m_utility[k][item_idx] * v)
        c += v
    if c == 0:
        return 0
    return s/c

# Creating Similarity Matrix

In [15]:
sim_mat_user = sim_matrix(mat)

In [16]:
# Testing the rated average of matrix with 
print(rec_via_users(mat,sim_mat_user,4,90,10))

-0.448275862069


This represents the difference between the mean ratings given by user and rating given by user for business id 90. So the recommendation system won't recommend this business to user as it is less than zero.

# Loading Test Data

In [17]:
res_mer_test = pd.read_csv('/media/gautam/DATA/desk_2104/CS Sem4/CS6220/Project/dataset/yelp.test.rating', sep ='\t')

# Prediction of average rating based on 10 similar users

In [18]:
pred_ratings = []
for index, row in res_mer_test.iterrows():
    pred = rec_via_users(mat, sim_mat_user, row.user_id, row.business_id, 10)
    pred_ratings.append(pred)

# Converting the test rating to average rating with mean zero to compare it with average predicted rating

In [19]:
test_ratings = res_mer_test.rating
arrMeanTest = np.mean(test_ratings)
test_ratings =  np.subtract(test_ratings, arrMeanTest)

In [20]:
#mean square error
arr_sub = np.subtract(pred_ratings,test_ratings)
arr_sqr = np.square(arr_sub)
arr_sum = np.sum(arr_sqr)
mse = np.mean(arr_sum)
print(mse)

34691.1133424


Model has not behaved as expected, there is large value of mean square error among the predicted average rating and real average rating for test data which show that model is not trained properly.

In [21]:
#Recommending Top 10 Business to particular user in training data
user_id = 0
pred_rating = {}
for i in range(len(mat[1])):
    if np.isnan(rating_mat[user_id][i]):
        pred_rating[i] = rec_via_users(mat, sim_mat_user, user_id, i, 10)
#sorted on basis of value
sorted_keys=sorted(pred_rating, key=pred_rating.get, reverse=True)

# List of Top 10 business id for user based on rating prediction

In [22]:
# Printing Top 10 Recommendation
c = 0
for r in sorted_keys:
    c = c + 1
    print(r)
    if c > 10:
        break

563
997
8258
22295
30668
1536
1676
1916
31060
1419
1420
